In [1]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW

In [2]:
Translation=pd.read_csv("/kaggle/input/arb-eng2/ara_eng.txt",delimiter="\t",names=["english","arabic"])

In [3]:
Translation.head()

,english,arabic
0,Hi.,مرحبًا.
1,Run!,اركض!
2,Help!,النجدة!
3,Jump!,اقفز!
4,Stop!,قف!


In [4]:
Translation.tail()

,english,arabic
24633,rising voices promoting a more linguistically ...,شاركنا تحدي ابداع ميم بلغتك الام تعزيزا للتنوع...
24634,following last year s successful campaign we i...,استكمالا لنجاح حملة العام السابق ندعوكم للمشار...
24635,during last year s challenge we also met langu...,تعرفنا خلال تحدي العام الماضي على ابطال لغويين...
24636,to take part just follow the simple steps outl...,للمشاركة في التحدي اتبع الخطوات الموضحة على ال...
24637,you will also find links to some free web base...,ستجد ايضا روابط لمجموعة من منصات ابداع الميم ا...


In [5]:
Translation.shape

(24638, 2)

In [6]:
Translation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24638 entries, 0 to 24637
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   english  24638 non-null  object
 1   arabic   24638 non-null  object
dtypes: object(2)
memory usage: 385.1+ KB


In [7]:
Translation.describe().T

,count,unique,top,freq
english,24638,22834,this post is part of our special coverage syri...,24
arabic,24638,24031,المواطن الرقمي الاصوات العالمية,11


In [8]:
input=Translation["arabic"].tolist()
target=Translation["english"].tolist()

In [9]:
input

['مرحبًا.',
 'اركض!',
 'النجدة!',
 'اقفز!',
 'قف!',
 'داوم.',
 'استمر.',
 'مرحباً.',
 'تعجّل!',
 'استعجل!',
 'انا اري',
 'أنا فُزت!',
 'استرح.',
 'ابتسم.',
 'في صحتك.',
 'هل فهمت؟',
 'ركض.',
 'أعرف.',
 'أعلم ذلك.',
 'أنا أعلم',
 'أنا في 19',
 'أنا بخير.',
 'استمع',
 'غير معقول!',
 'حقاً؟',
 'شكرا.',
 'لماذا أنا؟',
 'رائع!',
 'خذ راحتك.',
 'هاتفني.',
 'اتصل بي.',
 'تفضل بالدخول.',
 'تعال إلى الداخل',
 'بالله عليك!',
 'هيا',
 'هيّا',
 'اخرج من هنا!',
 'أُخرج!',
 'اخرج!',
 'اتركني و شأني.',
 'اذهب بعيداً.',
 'ارحل.',
 'مع السلامة.',
 'لقد أتى.',
 'هو يجري',
 'ساعدني!',
 'النجدة! ساعدني!',
 'أنا حزين.',
 'أنا أيضاً.',
 'اخرس!',
 'اصمت!',
 'اسكت!',
 'أغلق فمك!',
 'أوقفه',
 'خذه',
 'توم فاز.',
 'لقد ربح توم.',
 'استيقظ!',
 'أهلاً و سهلاً!',
 'مرحباً بك!',
 'اهلا وسهلا',
 'مرحبا!',
 'من فاز؟',
 'من الذي ربح؟',
 'لم لا؟',
 'لما لا؟',
 'استمتع بوقتك.',
 'أسرع!',
 'لقد نسيت.',
 'فهمتُهُ.',
 'فهمتُها.',
 'فَهمتُ ذلك.',
 'أستخدمه.',
 'سأدفع أنا.',
 'أنا مشغول.',
 'إنني مشغول.',
 'أشعر بالبرد.',
 '

In [10]:
target

['Hi.',
 'Run!',
 'Help!',
 'Jump!',
 'Stop!',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hurry!',
 'Hurry!',
 'I see.',
 'I won!',
 'Relax.',
 'Smile.',
 'Cheers!',
 'Got it?',
 'He ran.',
 'I know.',
 'I know.',
 'I know.',
 "I'm 19.",
 "I'm OK.",
 'Listen.',
 'No way!',
 'Really?',
 'Thanks.',
 'Why me?',
 'Awesome!',
 'Be cool.',
 'Call me.',
 'Call me.',
 'Come in.',
 'Come in.',
 'Come on!',
 'Come on!',
 'Come on!',
 'Get out!',
 'Get out!',
 'Get out.',
 'Go away.',
 'Go away.',
 'Go away.',
 'Goodbye!',
 'He came.',
 'He runs.',
 'Help me!',
 'Help me.',
 "I'm sad.",
 'Me, too.',
 'Shut up!',
 'Shut up!',
 'Shut up!',
 'Shut up!',
 'Stop it.',
 'Take it.',
 'Tom won.',
 'Tom won.',
 'Wake up!',
 'Welcome.',
 'Welcome.',
 'Welcome.',
 'Welcome.',
 'Who won?',
 'Who won?',
 'Why not?',
 'Why not?',
 'Have fun.',
 'Hurry up.',
 'I forgot.',
 'I got it.',
 'I got it.',
 'I got it.',
 'I use it.',
 "I'll pay.",
 "I'm busy.",
 "I'm busy.",
 "I'm cold.",
 "I'm free.",
 "I'm here.",
 "I'm hom

# Using model opus-mt-ar-en

In [11]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ar-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ar-en")


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [12]:
def encode_data(inputs, targets, tokenizer, max_length=128):
    # ترميز البيانات المدخلة (Arabic)
    input_encodings = tokenizer(inputs, truncation=True, padding=True, max_length=max_length, return_tensors="pt")

    # ترميز البيانات الهدف (English)
    target_encodings = tokenizer(targets, truncation=True, padding=True, max_length=max_length, return_tensors="pt")

    return input_encodings, target_encodings

# ترميز البيانات
input_encodings, target_encodings = encode_data(input, target, tokenizer)

In [13]:
print("Input Encodings:", input_encodings)
print("Target Encodings:", target_encodings)

Input Encodings: {'input_ids': tensor([[  111,   971,    27,  ..., 62833, 62833, 62833],
        [   23,  7354,   684,  ..., 62833, 62833, 62833],
        [ 1297,   188,  1383,  ..., 62833, 62833, 62833],
        ...,
        [ 1186,   114,   246,  ..., 62833, 62833, 62833],
        [ 8502,     9,  9786,  ..., 62833, 62833, 62833],
        [   23, 15606,  5059,  ..., 62833, 62833, 62833]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
Target Encodings: {'input_ids': tensor([[  817,   577,     2,  ..., 62833, 62833, 62833],
        [ 1061,   677,    65,  ..., 62833, 62833, 62833],
        [  817,  2938,  1312,  ..., 62833, 62833, 62833],
        ...,
        [18470,   113,  1465,  ..., 62833, 62833, 62833],
        [    8,    23,  1417,  ..., 62833, 62833, 62833],
        [ 2878,  4698,   115,  ...

In [14]:
class Translation_Dataset(Dataset):
    def __init__(self, input_encodings, target_encodings):
        self.input_encodings = input_encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.input_encodings['input_ids'])

    def __getitem__(self, idx):
        input_data = {key: val[idx].clone().detach() for key, val in self.input_encodings.items()}
        target_data = self.target_encodings['input_ids'][idx].clone().detach()
        return input_data, target_data

dataset = Translation_Dataset(input_encodings, target_encodings)


dataloader = DataLoader(dataset, batch_size=16, shuffle=True, pin_memory=True, num_workers=4)


In [15]:
for batch in dataloader:
    input_batch, target_batch = batch
    print("Input Batch:", input_batch)
    print("Target Batch:", target_batch)
    break  # للخروج بعد أول دفعة


Input Batch: {'input_ids': tensor([[   11,  4637,    77,  ..., 62833, 62833, 62833],
        [  126,   830,   114,  ..., 62833, 62833, 62833],
        [   16,    30,     9,  ..., 62833, 62833, 62833],
        ...,
        [  263,  9450,  1254,  ..., 62833, 62833, 62833],
        [   83,  1381,  3492,  ..., 62833, 62833, 62833],
        [  635,     9,    23,  ..., 62833, 62833, 62833]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
Target Batch: tensor([[  423,  1403,  8820,  ..., 62833, 62833, 62833],
        [   24,   205,  2878,  ..., 62833, 62833, 62833],
        [   25,    23,  2100,  ..., 62833, 62833, 62833],
        ...,
        [   23, 25591,    23,  ..., 62833, 62833, 62833],
        [  324,    60,  5407,  ..., 62833, 62833, 62833],
        [ 1153,  1795,  2592,  ..., 62833, 62833, 62833]

In [16]:
print("Input Batch Shape:", input_batch['input_ids'].shape)
print("Attention Mask Shape:", input_batch['attention_mask'].shape)
print("Target Batch Shape:", target_batch.shape)


Input Batch Shape: torch.Size([16, 128])
Attention Mask Shape: torch.Size([16, 128])
Target Batch Shape: torch.Size([16, 128])


In [17]:
dataset

In [18]:
dataloader

##Train_Model##

In [19]:
optimizer = AdamW(model.parameters(), lr=1e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
# إعداد الجهاز (CPU أو GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# إعداد معايير التدريب
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_function = torch.nn.CrossEntropyLoss()

# عدد العصور (epochs)
num_epochs = 10

# التدريب
for epoch in range(num_epochs):
    model.train()  # وضع التدريب
    total_loss = 0
    correct_predictions = 0  # لاحتساب التوقعات الصحيحة
    total_predictions = 0  # لاحتساب إجمالي التوقعات

    for batch in dataloader:
        input_data, target_data = batch
        input_data = {key: val.to(device) for key, val in input_data.items()}
        target_data = target_data.to(device)

        # الصفر خسارة النموذج
        optimizer.zero_grad()

        # تمرير البيانات عبر النموذج
        outputs = model(input_ids=input_data['input_ids'],
                        attention_mask=input_data['attention_mask'],
                        labels=target_data)

        loss = outputs.loss
        total_loss += loss.item()

        # حساب التوقعات
        logits = outputs.logits
        predicted_ids = torch.argmax(logits, dim=-1)

        # احتساب التوقعات الصحيحة
        correct_predictions += (predicted_ids == target_data).sum().item()
        total_predictions += target_data.numel()  # عدد العناصر في الدفعة

        # تحديث الأوزان
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(dataloader)
    accuracy = correct_predictions / total_predictions  # حساب الدقة

    print(f"Epoch: {epoch + 1}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")


Epoch: 1, Loss: 0.8042, Accuracy: 0.8436
Epoch: 2, Loss: 0.5198, Accuracy: 0.8878
Epoch: 3, Loss: 0.4370, Accuracy: 0.9028
Epoch: 4, Loss: 0.3830, Accuracy: 0.9127
Epoch: 5, Loss: 0.3409, Accuracy: 0.9207
Epoch: 6, Loss: 0.3069, Accuracy: 0.9271
Epoch: 7, Loss: 0.2777, Accuracy: 0.9329
Epoch: 8, Loss: 0.2515, Accuracy: 0.9382
Epoch: 9, Loss: 0.2289, Accuracy: 0.9428
Epoch: 10, Loss: 0.2081, Accuracy: 0.9470


In [21]:
model.save_pretrained("path_to_save_model")
tokenizer.save_pretrained("path_to_save_tokenizer")

/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('path_to_save_tokenizer/tokenizer_config.json',
 'path_to_save_tokenizer/special_tokens_map.json',
 'path_to_save_tokenizer/vocab.json',
 'path_to_save_tokenizer/source.spm',
 'path_to_save_tokenizer/target.spm',
 'path_to_save_tokenizer/added_tokens.json')

In [22]:
# تحميل النموذج والتوكنيزرالمسار الذي حفظت فيه النموذج
tokenizer = AutoTokenizer.from_pretrained ("/kaggle/working/path_to_save_tokenizer")
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/path_to_save_model")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # وضع التقييم

def translate(text):
    # ترميز النص المدخل
    inputs = tokenizer.encode("translate Arabic to English: " + text, return_tensors="pt").to(device)
    
    # توليد الترجمة
    outputs = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
    
    # فك ترميز المخرجات
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return translated_text

# اختبار النموذج بجمل مختلفة
test_sentences = [
    "مرحبا، كيف حالك؟",
    "ما هي عاصمة مصر؟",
    "أنا أحب تعلم البرمجة."
]

for sentence in test_sentences:
    translated = translate(sentence)
    print(f"النص المدخل: {sentence}")
    print(f"الترجمة: {translated}")
    print("-" * 30)



/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


النص المدخل: مرحبا، كيف حالك؟
الترجمة: Translate arabic to English: hi, how are you.
------------------------------
النص المدخل: ما هي عاصمة مصر؟
الترجمة: subtitle arabic to English what is the capital of egypt.
------------------------------
النص المدخل: أنا أحب تعلم البرمجة.
الترجمة: translation arabic to English: I like learning programing.
------------------------------


In [23]:
# عرض النص المدخل بعد الترميز والديكودينغ
input_ids = input_encodings['input_ids'][0]
decoded_input = tokenizer.decode(input_ids, skip_special_tokens=True)
print("Decoded Input:", decoded_input)

# عرض النص المترجم بعد الترميز والديكودينغ
target_ids = target_encodings['input_ids'][0]
decoded_target = tokenizer.decode(target_ids, skip_special_tokens=True)
print("Decoded Target:", decoded_target)


Decoded Input: مرحبًا.
Decoded Target: Hi.
